## Setup the environment

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sess = sage.Session()
model_package_arn='<REPLACE WITH MODEL ARN>'
s3_base_path='s3://<REPLACE WITH BUCKET NAME>'


## Realtime Inference

In [ ]:
from sagemaker import ModelPackage
model_name='people-tracking-model'
model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sess)

In [ ]:
endpoint_name = 'people-tracking-model'

In [ ]:
predictor = model.deploy(1, 'ml.m5.large', endpoint_name=endpoint_name)

In [ ]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sess
)

In [ ]:
# open s3 file
import io
import boto3
res = ""
s3_client = boto3.client('s3')
s3_response_object = s3_client.get_object(Bucket='<REPLACE WITH BUCKET NAME>', Key='input/sample.mp4')
object_content = s3_response_object['Body'].read()

res = predictor.predict(object_content)
print(len(res))

In [ ]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job 

In [ ]:
s3_output_path=s3_base_path + '/output/'
transformer = sage.transformer.Transformer(
    base_transform_job_name='people-tracking-model-batch-transform',
    model_name=model_name,
    max_payload= '<REPLACE WITH MAXIMUM SIZE OF INPUT VIDEO IN MB>',
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_path,
    sagemaker_session=sess
)

In [ ]:
transformer.transform(
    s3_base_path + '/input/'
)
# wait until prediction job is completed
transformer.wait()

#### Inspect the Batch Transform Output in S3